In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys 
sys.path.append('/home/hslee/Desktop/Embedded_AI/PyTorch_Tutorials/03_Pruning_Filters_for_Efficient_Convnets/')
from utils import *
from architecture import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
val_loader, tesize = loadValDataset()

Files already downloaded and verified
# test datas : 10000


# vgg-16

In [12]:
model = VGG16_BN()
model.load_state_dict(torch.load('./00_vgg16_baseline_exp1/checkpoint/best_model.pth')['model_state_dict'])
top1_acc, top5_acc = testAccuracy(model, val_loader)
num_params = sum(p.numel() for p in model.parameters())


print(f"Top-1 Accuracy : {top1_acc:.2f} %")
print(f"# Parameters : {num_params}")

# number of flops
! flopth -m VGG16_BN -p ./architecture -i 3 32 32

Top-1 Accuracy : 91.86 %
# Parameters : 14991946
Op Flatten is not supported at now, set FLOPs of it to zero.
+---------------+---------------+-------------+-------------+------------------------------+----------+------------------+----------------------+----------+-----------------+---------------------+
| module_name   | module_type   | in_shape    | out_shape   | kernel_size,padding,stride   | params   | params_percent   | params_percent_vis   | flops    | flops_percent   | flops_percent_vis   |
+===============+===============+=============+=============+==============================+==========+==================+======================+==========+=================+=====================+
| conv1.0       | Conv2d        | (3,32,32)   | (64,32,32)  | k=(3, 3), p=(1, 1), s=(1, 1) | 1.792K   | 0.0119531%       |                      | 1.83501M | 0.583105%       |                     |
+---------------+---------------+-------------+-------------+------------------------------+----------

# vgg-16-pruned-A

In [13]:
model = VGG16_BN_PRUNE_FOR_SCRATCH()
model.load_state_dict(torch.load('./01_vgg16_prune_A/checkpoint/best_model.pth')['model_state_dict'])
top1_acc, top5_acc = testAccuracy(model, val_loader)
num_params = sum(p.numel() for p in model.parameters())


print(f"Top-1 Accuracy : {top1_acc:.2f} %")
print(f"# Parameters : {num_params}")

# number of flops
! flopth -m VGG16_BN_PRUNE_FOR_SCRATCH -p ./architecture -i 3 32 32

Top-1 Accuracy : 91.47 %
# Parameters : 5399690
Op Flatten is not supported at now, set FLOPs of it to zero.
+---------------+---------------+-------------+-------------+------------------------------+----------+------------------+----------------------+----------+-----------------+---------------------+
| module_name   | module_type   | in_shape    | out_shape   | kernel_size,padding,stride   | params   | params_percent   | params_percent_vis   | flops    | flops_percent   | flops_percent_vis   |
+===============+===============+=============+=============+==============================+==========+==================+======================+==========+=================+=====================+
| conv1.0       | Conv2d        | (3,32,32)   | (32,32,32)  | k=(3, 3), p=(1, 1), s=(1, 1) | 896.0    | 0.0165935%       |                      | 917.504K | 0.442566%       |                     |
+---------------+---------------+-------------+-------------+------------------------------+----------+

In [17]:
num_flops_before_pruning = 314696000
num_flops_after_pruning = 207314000

print(f"Pruned : {(num_flops_before_pruning - num_flops_after_pruning) / num_flops_before_pruning * 100:.2f}%")

Pruned : 34.12%


In [18]:
num_params_before_pruning = 14991900
num_params_after_pruning =   5399690

print(f"Pruned : {round((num_params_before_pruning - num_params_after_pruning) / num_params_before_pruning * 100):.2f}%")

Pruned : 64.00%


# vgg-16-pruned-A scratch-train

In [14]:
model = VGG16_BN_PRUNE_FOR_SCRATCH()
model.load_state_dict(torch.load('./02_vgg16_prune_A_scratch/checkpoint/best_model.pth')['model_state_dict'])
top1_acc, top5_acc = testAccuracy(model, val_loader)
num_params = sum(p.numel() for p in model.parameters())

print(f"Top-1 Accuracy : {top1_acc:.2f} %")
print(f"# Parameters : {num_params}")

# number of flops
! flopth -m VGG16_BN_PRUNE_FOR_SCRATCH -p ./architecture -i 3 32 32

Top-1 Accuracy : 88.68 %
# Parameters : 5399690
Op Flatten is not supported at now, set FLOPs of it to zero.
+---------------+---------------+-------------+-------------+------------------------------+----------+------------------+----------------------+----------+-----------------+---------------------+
| module_name   | module_type   | in_shape    | out_shape   | kernel_size,padding,stride   | params   | params_percent   | params_percent_vis   | flops    | flops_percent   | flops_percent_vis   |
+===============+===============+=============+=============+==============================+==========+==================+======================+==========+=================+=====================+
| conv1.0       | Conv2d        | (3,32,32)   | (32,32,32)  | k=(3, 3), p=(1, 1), s=(1, 1) | 896.0    | 0.0165935%       |                      | 917.504K | 0.442566%       |                     |
+---------------+---------------+-------------+-------------+------------------------------+----------+